# Лабораторная работа 5: Обучение и тестирование модели

Для начала выполним импорт библиотек

In [ ]:
import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import zipfile
from PIL import Image
import glob
from sklearn.model_selection import train_test_split
from typing import List, Tuple, Any

Читаем пути из аннотаций формата csv

In [ ]:
cat_dframe = pd.read_csv(cat_csv_path, delimiter=",", names=["Absolute path", "Relative path", "Class"])
    dog_dframe = pd.read_csv(dog_csv_path, delimiter=",", names=["Absolute path", "Relative path", "Class"])

    cat_images = cat_dframe["Absolute path"].tolist()
    dog_images = dog_dframe["Absolute path"].tolist()

    return cat_images, dog_images

img in index 0: The absolute path

img in index 1: C:/Users/User/Desktop/testing/dataset\cat\cat.4001.jpg

img in index 2: C:/Users/User/Desktop/testing/dataset\cat\cat.4002.jpg

img in index 0: The absolute path

img in index 1: C:/Users/User/Desktop/testing/dataset\dog\dog.4001.jpg

img in index 2: C:/Users/User/Desktop/testing/dataset\dog\dog.4002.jpg

Разделение данных на тренировочные, тестовые и валидационные в пропорциях: 80 на 10 на 10, соответственно

In [ ]:
cat_train_data, cat_test_val_data = train_test_split(cat_images, test_size=(test_size + val_size), random_state=42)
    cat_test_data, cat_valid_data = train_test_split(cat_test_val_data, test_size=(val_size / (test_size + val_size)), random_state=42)

    dog_train_data, dog_test_val_data = train_test_split(dog_images, test_size=(test_size + val_size), random_state=42)
    dog_test_data, dog_valid_data = train_test_split(dog_test_val_data, test_size=(val_size / (test_size + val_size)), random_state=42)

    train_data = cat_train_data + dog_train_data
    test_data = cat_test_data + dog_test_data
    valid_data = cat_valid_data + dog_valid_data

    return train_data, test_data, valid_data

1619
202
204

Проверка изображений

In [ ]:
random_idx = np.random.randint(1,len(img_list),size=10)

fig = plt.figure()
i=1
for idx in random_idx:
    ax = fig.add_subplot(2,5,i)
    img = Image.open(img_list[idx])
    plt.imshow(img)
    i+=1
    plt.axis('off')


plt.show()

![Alt text](fig1.png)
![Alt text](fig2.png)

Класс для хранения датасета

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data : List[str], labels : List[int], transform : Any = None):
        """
        create custom dataset
        Parameters
        ----------
        data : List[str]
        labels : List[int]
        transform : Any, optional
        """
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self) -> int:
        """
        get the length of dataset
        """
        return len(self.data)

    def __getitem__(self, index : int) -> Tuple[torch.Tensor, int]:
        """
        method for get img and him label
        Parameters
        ----------
        index : int
        """
        img_path = self.data[index]
        img = Image.open(img_path)
        img = self.transform(img) if self.transform else transforms.ToTensor()(img)
        label = self.labels[index]
        return img, label

Здесь мы подготавливаем размеры картинок для дальнейшего обучения

In [ ]:
custom_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

Проверка размера картинок:

In [ ]:
sample_img, sample_label = trenkaset[0]
    print(f"Shape of an image: {sample_img.shape}, Label: {sample_label}")

Shape of an image: torch.Size([3, 224, 224]), Label: 0

Теперь создаем класс сверточной модели нейронной сети:
1. 3-и слоя свертки и 2-а полностью связанных слоя;
2. пакетная нормализация для ограничения обучения.

In [ ]:
class Cnn(nn.Module):
    """
    Convolutional neural network model
    """

    def __init__(self) -> None:
        super(Cnn, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size = 3, padding = 0, stride = 2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size = 3, padding = 0, stride = 2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, padding = 0, stride = 2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.fc1 = nn.Linear(3*3*64,10)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(10, 2)
        self.relu = nn.ReLU()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1234)
if device =='cuda':
    torch.cuda.manual_seed_all(1234)
    
model = Cnn()
model.train()

In [ ]:
Cnn(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=576, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=10, out_features=2, bias=True)
  (relu): ReLU()
)

Функция, отрисовывающая графики:

In [ ]:
def show_results(epochs : int, acc : List[float], loss : List[float], v_acc : List[float], v_loss : List[float]) -> None:
    """
    creates graphs based on the learning results for train and validation sets.
    Parameters
    ----------
    epochs : int
    acc : List[float]
    loss : List[float]
    v_acc : List[float]
    v_loss : List[float]
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    ax1.plot(range(epochs), acc, color="orange", label="Train accuracy")
    ax2.plot(range(epochs), loss, color="orange", label="Train loss")
    
    ax1.plot(range(epochs), v_acc, color="steelblue", label="Validation accuracy")
    ax2.plot(range(epochs), v_loss, color="steelblue", label="Validation loss")
    
    ax1.legend()
    ax2.legend()
    plt.show()

Цикл обучения

In [ ]:
def train_loop(epochs: int, batch_size: int, lear: float, train_data: CustomDataset, test_data: CustomDataset, valid_data: CustomDataset) -> Tuple[list, Cnn]:
    optimizer = optim.Adam(params=model.parameters(), lr=lear)
    criterion = nn.CrossEntropyLoss()

    accuracy_values = []
    loss_values = []
    valid_accuracy_values = []
    valid_loss_values = []

    valid_loader = torch.utils.data.DataLoader(
        dataset=valid_data, batch_size=batch_size, shuffle=False
    )

    train_loader = torch.utils.data.DataLoader(
        dataset=train_data, batch_size=batch_size, shuffle=True
    )

    for epoch in range(epochs):
        epoch_loss = 0
        epoch_accuracy = 0

        for data, label in train_loader:
            data = data.to(device)
            label = label.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, label)

            loss.backward()
            optimizer.step()

            acc = (output.argmax(dim=1) == label).float().mean()
            epoch_accuracy += acc / len(train_loader)
            epoch_loss += loss / len(train_loader)

        print(f"Epoch : {epoch + 1}, train accuracy : {epoch_accuracy}, train loss : {epoch_loss}")
        accuracy_values.append(epoch_accuracy.item())
        loss_values.append(epoch_loss.item())
    with torch.no_grad():
            model.eval()
            epoch_val_accuracy = 0
            epoch_val_loss = 0

            for data, label in valid_loader:
                data = data.to(device)
                label = label.to(device)

                val_output = model(data)
                val_loss = criterion(val_output, label)

                acc = (val_output.argmax(dim=1) == label).float().mean()
                epoch_val_accuracy += acc / len(valid_loader)
                epoch_val_loss += val_loss / len(valid_loader)

            print(f"Epoch : {epoch + 1}, val_accuracy : {epoch_val_accuracy}, val_loss : {epoch_val_loss}")
            valid_accuracy_values.append(epoch_val_accuracy.item())
            valid_loss_values.append(epoch_val_loss.item())

Делаем 9 запусков с разными параметрами(эпохи, батчи, и скорость обучения)

Training with epochs=5, batch_size=32, learning_rate=0.001
![Alt text](1launch.png)
Training with epochs=10, batch_size=64, learning_rate=0.001
![Alt text](2launch.png)
Training with epochs=20, batch_size=256, learning_rate=0.001
![Alt text](3launch.png)
Training with epochs=25, batch_size=256, learning_rate=0.005
![Alt text](4launch.png)
Training with epochs=25, batch_size=512, learning_rate=0.005
![Alt text](5launch.png)
Training with epochs=30, batch_size=512, learning_rate=0.015
![Alt text](6launch.png)
Training with epochs=40, batch_size=512, learning_rate=0.015
![Alt text](7launch.png)
Training with epochs=50, batch_size=1024, learning_rate=0.015
![Alt text](8launch.png)
Training with epochs=15, batch_size=128, learning_rate=0.1
![Alt text](9launch.png)

На мой взгляд, для сохранения лучше использовать 4 модель, потому что у неё более менее стабильные маленькие потери.

Теперь заново запускаем и сохраняем csv файл и модель

In [ ]:
def save_result(rose_probs : List[Tuple[int, float]], csv_path : str, model : nn.Module, model_path : str) -> None:
    """
    func for saving the result in csv and the model.

    Parameters
    ----------
    rose_probs : List[Tuple[int, float]]
    csv_path : str
    model : nn.Module
    model_path : str
    """
    idx = list(i for i in range(len(rose_probs)))
    prob = list(map(lambda x: x[1], rose_probs))
    submission = pd.DataFrame({"id": idx, "label": prob})
    submission.to_csv(csv_path, index=False)

    torch.save(model.state_dict(), model_path)

Теперь повторно инициализируем модель и тестируем ее:

In [ ]:
new_model = Cnn().to(device)

In [ ]:
class_mapping = {0: "cat", 1: "dog"}

    fig, axes = plt.subplots(1, 5, figsize=(20, 12), facecolor="w")
    submission = pd.read_csv('E:\\education\\pythonLab\\pythonLabs\\Lab5\\result.csv')

    for ax in axes.ravel():
        i = random.choice(submission["id"].values)
        label = submission.loc[submission["id"] == i, "label"].values[0]
        if label > 0.5:
            label = 1
        else:
            label = 0

        img_path = testikset.data[i]
        img = Image.open(img_path)
        img_tensor = testikset.transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = loaded_model(img_tensor)

        predicted_label = torch.argmax(output).item()

        ax.set_title(f"True: {class_mapping[label]}, Predicted: {class_mapping[predicted_label]}")
        ax.imshow(img)
    
    plt.show()

Результаты:

![Alt text](result.png)